
# Projeto 2 - SCC0252 (Visualização Computacional)


*   Otávio Ferracioli Coletti - 11767796
*   Fernando César Lopes Barbosa Filho - 10260559

\\

---

In [1]:
# Bibliotecas importadas
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import Button, Layout
from matplotlib.spines import rcParams
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib import legend
from pandas import Series
import ipywidgets
import random
import plotly
import io

In [2]:
# Conjunto de dados do Spotify
url = 'https://raw.githubusercontent.com/otaviofcoletti/VIZCOMP/main/SpotifyFeatures(1).csv'
df = pd.read_csv(url)

In [3]:
popular_music_rank = df.sort_values('popularity', ascending=False) 

## Gráfico de Radar

Nesse tipo de visualização o usuário poderá selecionar quantas músicas se deseja analisar entre as mais populares, quais as *features* a serem os eixos e a respectiva ordem entre elas.

In [4]:
cinza = "#808080"
transparente = "rgba(0,0,0,0)"

def radar_chart_2(number_of_music=1,**dd_list):

  top5 = popular_music_rank[:number_of_music]

  lista_features = ['Feature 1','Feature 2','Feature 3','Feature 4','Feature 5','Feature 6','Feature 7','Feature 8']
  categories = ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']
  top5 = top5[['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness','track_name','artist_name']]
  fig = go.Figure()
  features = []

  for i in lista_features:

    if dd_list[i] != "Nenhum":
      features.append(dd_list[i])
  

  for i in range(0,number_of_music):

    fig.add_trace(go.Scatterpolar(
          r=list(top5.iloc[i]),
          theta=features,
          fill='toself',
          name= top5.iloc[i]['track_name'],
          legendgrouptitle_text="Músicas mais populares"
    ))


  fig.update_layout(
        font_color=cinza,
        font_size=15,
        autosize=True,
        height=400,
        margin=dict(l=40, r=40, t=25, b=25),
        paper_bgcolor=transparente,
        polar=dict(
            bgcolor=transparente,
            radialaxis=dict(
                range=[0, 1],
                linecolor=cinza,
                gridcolor=cinza,
                tickcolor=cinza,
            ),
            angularaxis=dict(
                rotation=120,
                direction="clockwise",
                linecolor=cinza,
                tickcolor=cinza,
                gridcolor=cinza,
            ),
        ),
    )
  
  fig.show()

In [5]:
lista_features = ['Feature 1','Feature 2','Feature 3','Feature 4','Feature 5','Feature 6','Feature 7','Feature 8']

In [6]:
charac=['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness','Nenhum']

In [7]:
index_list = [0,1,2,3,4,5,6,7,8]
random.seed(117)

dd_radar = {descri: widgets.Dropdown(
    options=charac,
    value=charac[index],
    description=descri,
    disabled=False,
) for index,descri in zip(index_list,lista_features) }

Através do gráfico de radar interativo podemos escolher quais as *features* musicais de interesse na análise, bem como escolher quantas e quais músicas comparar clicando na legenda associada. O usuário poderá escolher até 8 features, mas recomendamos entre 5 e 6 para facilitar a visualização.

In [8]:
ipywidgets.interactive(radar_chart_2, number_of_music=ipywidgets.IntSlider(min=1,max=15,step=1,value=5),**dd_radar)

interactive(children=(IntSlider(value=5, description='number_of_music', max=15, min=1), Dropdown(description='…

Podemos ver que, das músicas mais populares, todas possuem um *loudness* alto e uma *danceability* significativamente alta também. Notamos que, por exemplo, a acústica dessas músicas dificilmente ultrapassa 5. \\

Portanto, para ter sucesso é preciso fazer barulho, fazer o pessoal dançar e usar bastante da eletrônica na sua música! \\


## Heatmap

Geramos um *heatmap* que exibe os valores de correlação entre os atributos de todas as músicas do Spotify.


In [9]:
popular_music_rank = df.sort_values('popularity', ascending=False) 

In [10]:
def heatmap_interactive(**ch_list):

  categories = ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']
  features = []

  for i in categories:
    if ch_list[i]:
      features.append(i)

  selection = df[features]
  plt.figure(figsize=(15,10))
  sns.heatmap(selection.corr(),annot=True,cmap="coolwarm")
  plt.title('Heatmap of Correlations')
  plt.show()

In [11]:
cb_list = {description: widgets.Checkbox(
    value=random.choice([True, False]),
    description=description,
    disabled=False,
    indent=False
) for description in ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']}

Selecione nos **checkboxes** as *features* nas quais se deseja comparar a correlação no **heatmap**:

In [12]:
widgets.interactive(heatmap_interactive, **cb_list)

interactive(children=(Checkbox(value=True, description='valence', indent=False), Checkbox(value=True, descript…